# Train All Models

In [4]:
import pickle
import leaderbot as lb
import numpy as np

In [5]:
data = lb.data.load()
training_data, test_data = lb.data.split(data, test_ratio=0.1, seed=20)

In [6]:
from copy import deepcopy

# Create a copy of the data with no tie (for Bradley-Terry)
training_data_no_tie = deepcopy(training_data)
test_data_no_tie = deepcopy(test_data)

training_data_no_tie['Y'][:, -1] = 0
test_data_no_tie['Y'][:, -1] = 0

In [13]:
# More set of models
n_tie = 0
n_cov = 0

models = [
    lb.models.BradleyTerry(training_data),
    lb.models.BradleyTerryScaled(training_data),
    #lb.models.BradleyTerryScaledR(training_data_no_tie),
    # lb.models.BradleyTerryScaledRIJ(training_data, n_tie_factors=n_tie),
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=1),
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=5),
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=10),

    lb.models.BradleyTerry(training_data_no_tie),
    lb.models.BradleyTerryScaled(training_data_no_tie),
    lb.models.BradleyTerryFactor(training_data_no_tie, n_cov_factors=1),
    lb.models.BradleyTerryFactor(training_data_no_tie, n_cov_factors=5),
    lb.models.BradleyTerryFactor(training_data_no_tie, n_cov_factors=10),
    
    lb.models.RaoKupper(training_data, n_tie_factors=n_tie),
    lb.models.RaoKupperScaled(training_data, n_tie_factors=n_tie),
    #lb.models.RaoKupperScaledR(training_data, n_tie_factors=n_tie),
    # lb.models.RaoKupperScaledRIJ(dtraining_data, n_tie_factors=n_tie),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=1, n_tie_factors=0),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=1, n_tie_factors=0),
    
    # lb.models.Davidson(training_data, n_tie_factors=n_tie),
    # lb.models.DavidsonScaled(training_data, n_tie_factors=n_tie),
    # #lb.models.DavidsonScaledR(training_data, n_tie_factors=n_tie),
    # # lb.models.DavidsonScaledRIJ(training_data, n_tie_factors=n_tie),
    # lb.models.DavidsonFactor(training_data, n_cov_factors=0, n_tie_factors=0)
]

In [14]:
# Train all. If model is ScaledRIJ, use l-BFGS-B, otherwise, use BFGS.
for model in models:

    name = model.__class__.__name__
    if name.endswith('ScaledRIJ'):
        method = 'L-BFGS-B'
    else:
        method = 'BFGS'

    print(f'training {name:<21} ... ', end='')
    model.train()
    print('done.')

training BradleyTerry          ... done.
training BradleyTerry          ... done.
training BradleyTerryScaled    ... done.
training BradleyTerryScaled    ... done.
training BradleyTerryFactor    ... done.
training BradleyTerryFactor    ... done.
training BradleyTerryFactor    ... done.
training BradleyTerryFactor    ... done.
training BradleyTerryFactor    ... done.
training BradleyTerryFactor    ... done.


In [15]:
with open('models.pkl', 'wb') as f:
    pickle.dump(models, f)

In [16]:
# Model selection
metrics = lb.evaluate.model_selection(models, train=False, tie=True, report=True)

+-----------------------+---------+--------+--------+---------+---------+
| model                 | # param | NLL    | CEL    | AIC     | BIC     |
+-----------------------+---------+--------+--------+---------+---------+
| BradleyTerry          |     129 | 0.6552 |    inf |  256.69 | 1036.11 |
| BradleyTerry          |     129 | 0.6347 | 0.6347 |  256.73 | 1036.16 |
| BradleyTerryScaled    |     258 | 0.6550 |    inf |  514.69 | 2073.54 |
| BradleyTerryScaled    |     258 | 0.6343 | 0.6343 |  514.73 | 2073.58 |
| BradleyTerryFactor    |     387 | 0.6549 |    inf |  772.69 | 3110.97 |
| BradleyTerryFactor    |     387 | 0.6340 | 0.6340 |  772.73 | 3111.01 |
| BradleyTerryFactor    |     645 | 0.6547 |    inf | 1288.69 | 5185.82 |
| BradleyTerryFactor    |     645 | 0.6338 | 0.6338 | 1288.73 | 5185.86 |
| BradleyTerryFactor    |     903 | 0.6546 |    inf | 1804.69 | 7260.67 |
| BradleyTerryFactor    |     903 | 0.6335 | 0.6335 | 1804.73 | 7260.71 |
+-----------------------+---------+---

In [19]:
# Model selection
metrics = lb.evaluate.model_selection(models, train=False, tie=False, report=True)

+-----------------------+---------+--------+--------+---------+---------+
| model                 | # param | NLL    | CEL    | AIC     | BIC     |
+-----------------------+---------+--------+--------+---------+---------+
| BradleyTerry          |     129 | 0.6552 | 0.6368 |  256.69 | 1036.11 |
| BradleyTerry          |     129 | 0.6347 | 0.6347 |  256.73 | 1036.16 |
| BradleyTerryScaled    |     258 | 0.6550 | 0.6364 |  514.69 | 2073.54 |
| BradleyTerryScaled    |     258 | 0.6343 | 0.6343 |  514.73 | 2073.58 |
| BradleyTerryFactor    |     387 | 0.6549 | 0.6363 |  772.69 | 3110.97 |
| BradleyTerryFactor    |     387 | 0.6340 | 0.6340 |  772.73 | 3111.01 |
| BradleyTerryFactor    |     645 | 0.6547 | 0.6360 | 1288.69 | 5185.82 |
| BradleyTerryFactor    |     645 | 0.6338 | 0.6338 | 1288.73 | 5185.86 |
| BradleyTerryFactor    |     903 | 0.6546 | 0.6358 | 1804.69 | 7260.67 |
| BradleyTerryFactor    |     903 | 0.6335 | 0.6335 | 1804.73 | 7260.71 |
+-----------------------+---------+---

In [17]:
# Goodness of fit. This evaluates how well the model fits training data.
metrics = lb.evaluate.goodness_of_fit(models, train=False, tie=True, metric='MAE', 
                                      density=True, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |             MAE            |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.13  10.22  17.91  12.76 |    inf | 7.2576 |
| BradleyTerry          |  4.98   4.98   0.00   3.32 | 0.0183 | 0.5276 |
| BradleyTerryScaled    |  9.98  10.20  17.91  12.70 |    inf | 7.2299 |
| BradleyTerryScaled    |  4.66   4.66   0.00   3.11 | 0.0171 | 0.4960 |
| BradleyTerryFactor    |  9.98  10.13  17.91  12.67 |    inf | 7.2222 |
| BradleyTerryFactor    |  4.52   4.52   0.00   3.01 | 0.0167 | 0.4859 |
| BradleyTerryFactor    |  9.85  10.05  17.91  12.61 |    inf | 7.1924 |
| BradleyTerryFactor    |  4.25   4.25   0.00   2.83 | 0.0155 | 0.4565 |
| BradleyTerryFactor    |  9.86  10.01  17.91  12.59 |    inf | 7.1813 |
| BradleyTerryFactor    |  3.98   3.98   0.00   2.6

In [18]:
# Generalization. This evaluates how well the model predicts test data.
metrics = lb.evaluate.generalization(models, test_data=test_data, train=False,
                                     tie=True, metric='MAE', density=True,
                                     report=True)

+-----------------------+----------------------------+--------+--------+
|                       |             MAE            |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.45   9.74  17.63  12.61 |    inf | 7.0958 |
| BradleyTerry          | 11.02   9.36  17.63  12.67 |    inf | 7.0616 |
| BradleyTerryScaled    | 10.36   9.71  17.63  12.56 |    inf | 7.0897 |
| BradleyTerryScaled    | 10.93   9.41  17.63  12.66 |    inf | 7.0650 |
| BradleyTerryFactor    | 10.32   9.80  17.63  12.58 |    inf | 7.0863 |
| BradleyTerryFactor    | 10.89   9.54  17.63  12.69 |    inf | 7.0635 |
| BradleyTerryFactor    | 10.49   9.91  17.63  12.68 |    inf | 7.1103 |
| BradleyTerryFactor    | 11.05   9.73  17.63  12.80 |    inf | 7.1141 |
| BradleyTerryFactor    | 10.83  10.05  17.63  12.84 |    inf | 7.1955 |
| BradleyTerryFactor    | 11.31   9.79  17.63  12.9